# 概要
* 複数インデックスの動作の再検証を行う
* インデックスの分割単位を小さくする
* そもそも論として、まとめからまとめようとしているから良くないのでは？？
* チャンクサイズを落とすと精度が向上する
* ただし当たり前ながら、区切り文字以下のチャンクサイズを指定するとエラーとなる
* 結局動的に読み込む機構は現在のところない
* 例えば、DBなら「〜編」で区切ると精度が上がる（ルールベース）
* オブジェクト指向がわからないと、ライブラリ以上の精度は出せない

# 参考
* https://note.com/npaka/n/n5d4ff4380055

In [1]:
import os
import openai
import tiktoken

#from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader, GPTTreeIndex, GPTListIndex, GPTSimpleKeywordTableIndex
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader
)

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
documents1 = SimpleDirectoryReader('story/dragonball').load_data()

In [4]:
type(documents1), len(documents1), type(documents1[0]), documents1[0].text,

(list,
 1,
 llama_index.readers.schema.base.Document,
 '孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nピッコロ大魔王編\n天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。\n\nその約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初

In [5]:
documents2 = SimpleDirectoryReader('story/dai').load_data()

In [6]:
documents2[0].text

'序章（デルパ! イルイル! / ダイ爆発!!!）\n魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。\n\nそれから十数年後。多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年ダイは、島唯一の人間として、ゴールデンメタルスライムのゴメちゃんを始めとする友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中でも何度かの冒険や騒動もあり、その過程でダイはパプニカ王国王女のレオナと知り合う。\n\nダイの旅立ち\nある日、ゴメちゃん以外の島のモンスターたちが凶暴化。辛うじて理性を保っているブラスは魔王が復活したことを察する。そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはレオナからダイの事を聞き、その将来性を見込んで島を訪れたのである。アバンの弟子となったダイは、魔王を倒し平和を取り戻すべく、アバンの指導を受けて、短期間のうちに秘められた力を開花させていく。\n\n修行の最中、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。アバンこそが、かつて魔王ハドラーを倒した勇者だったのである。現在のハドラーは「魔界の神」を称する大魔王バーンの力で蘇り、再び地上を制圧するために編成された魔王軍の総司令となっていた。バーンによって力を増したハドラーはアバンを圧倒する。アバンは弟子たちを護るために自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残り、ダイ達に牙をむく。しかし、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共にデルムリン島から旅立っていく。\n\n魔王軍六大軍団との戦い\nロモス王国に上陸したダイ一行は、同じアバンの弟子である僧侶戦士マァムを仲間に加え、王国を占拠した魔王軍幹部の獣王クロコダインと戦い、これを打ち破る。続いて訪れたパプニカ王国では魔王軍幹部であると同時にアバンの一番弟子でもある魔剣戦士ヒュンケルと激闘を繰り広げるが、ダイとポップの姿に心を打たれたクロコ

# GPT-indexを用いた場合

In [18]:
index1 = GPTSimpleVectorIndex(
    documents1,  # ドキュメント
    chunk_size_limit=512
)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4415 tokens


In [9]:
index2 = GPTSimpleVectorIndex(
    documents2,  # ドキュメント
    chunk_size_limit=1024
)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4410 tokens


In [10]:
type(index1),

(llama_index.indices.vector_store.vector_indices.GPTSimpleVectorIndex,)

In [20]:
index1._index_struct.embeddings_dict

{'bbf07657-0fc3-4b96-b785-e322937825a3': [0.009834830649197102,
  -0.019985215738415718,
  -0.013457151129841805,
  -0.008342512883245945,
  -0.009065662510693073,
  0.014265763573348522,
  -0.016159098595380783,
  -0.029846342280507088,
  -0.025783559307456017,
  -0.01633002609014511,
  0.008427976630628109,
  0.01222122274339199,
  0.01999836415052414,
  0.008178160525858402,
  0.0025474578142166138,
  -0.012582797557115555,
  0.01769743487238884,
  -0.026296338066458702,
  0.024403002113103867,
  -0.01515983883291483,
  0.02840004488825798,
  -0.013976503163576126,
  -0.0015219005290418863,
  -0.0014150716597214341,
  -0.010137238539755344,
  0.0008858578512445092,
  0.03400116413831711,
  -0.024389853700995445,
  0.029451897367835045,
  -0.010669738985598087,
  0.020432254299521446,
  0.007428715471178293,
  0.0009532422409392893,
  0.002586902119219303,
  -0.011175943538546562,
  -0.002184239448979497,
  0.0022713460493832827,
  0.0003654780157376081,
  0.011478351429104805,
  -0.

In [46]:
#dir(index1)
#index1.docstore
#dir(index1.docstore)
#dir(index1.docstore.docs)
index1.docstore.docs.keys()
index1.docstore.docs.values()

dict_values([SimpleIndexDict(text=None, doc_id='4a0f42a4-b95e-45fe-8051-fb162687af9d', embedding=None, extra_info=None, nodes_dict={1644020277721860963: Node(text='孫悟空少年編 地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。 ', doc_id='42591acc-8a9b-43bd-999d-1e914c945418', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='a808ea23-c2eb-49f8-8571-53aea5d29e92', node_info={'start': 0, 'end': 298}), 2883464449922835992: Node(text=' その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。  ピッコロ大魔王編', doc_id='9c354a75-2724-4b3a-b075-1

# 各nodeに要約の設定

In [11]:
summary = index1.query("宇宙の帝王とは誰ですか？")
summary

INFO:root:> [query] Total LLM token usage: 413 tokens
INFO:root:> [query] Total embedding token usage: 18 tokens


Response(response='\n宇宙の帝王とは、フリーザです。', source_nodes=[SourceNode(source_text=' フリーザ編 地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。だが、そこには地球で闘ったベジータや、界王すら畏怖する宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ナメック星人を虐殺しながらドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。 ', doc_id='a808ea23-c2eb-49f8-8571-53aea5d29e92', extra_info=None, node_info={'start': 1352, 'end': 1618}, similarity=0.8299279132219018)], extra_info=None)

In [15]:
summary = index1.query("フリーザは何回変身しますか？")
summary

INFO:root:> [query] Total LLM token usage: 525 tokens
INFO:root:> [query] Total embedding token usage: 21 tokens


Response(response='\nフリーザは2回変身します。最初は圧倒的な力を持つ力を解放した状態で、2回目はナメック星の爆発を脱出した後にヤードラット星に漂着した状態で変身します。', source_nodes=[SourceNode(source_text=' 出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる。フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。  人造人間・セル編', doc_id='a808ea23-c2eb-49f8-8571-53aea5d29e92', extra_info=None, node_info={'start': 1624, 'end': 1880}, similarity=0.8345543993588449)], extra_info=None)

In [ ]:
index1.set_text(summary.response)

In [ ]:
summary = index2.query("内容を要約してください。")
summary

In [ ]:
index2.set_text(summary.response)

In [ ]:
list_index = GPTListIndex([index1, index2])
#keyword_table = GPTSimpleKeywordTableIndex([index2], max_keywords_per_chunk=50)

# 　質問

In [ ]:
# set query config
query_configs = [
    {
        "index_struct_type": "simple_dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 1
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "simple",
        "query_kwargs": {}
    },
]

In [ ]:
response = list_index.query(
    "ダイの友達は誰ですか？",
    mode="recursive",
    query_configs=query_configs
)

In [ ]:
response

In [ ]:
response = list_index.query(
    "フリーザを倒したのは誰ですか？",
    mode="recursive",
    query_configs=query_configs
)

In [ ]:
response

# Keywordの利用
* そもそもkeywordが日本語だから拾えてない？

In [ ]:
keyword_table = GPTSimpleKeywordTableIndex([index1, index2], max_keywords_per_chunk=50)

In [ ]:
from llama_index.composability import ComposableGraph

In [ ]:
graph = ComposableGraph.build_from_index(keyword_table)

In [ ]:
graph.save_to_disk("index_graph.json")

In [ ]:
import json

f = open('index_graph.json', 'r')
json_dict = json.load(f)
print(json_dict)

In [ ]:
graph = ComposableGraph.load_from_disk("index_graph.json")

In [ ]:
response = graph.query(
    "サイヤ人編のあらすじを教えて", 
    query_configs=query_configs
)

In [ ]:
response